## Set up HaskellR

In [1]:
:ext QuasiQuotes
import qualified H.Prelude as H
H.initialize H.defaultConfig

## Doing things in R

In [ ]:
-- print output
[rprint| lm(mpg ~ wt, data = mtcars) |]

In [ ]:
-- plot stuff
-- we just require ggplot2 as we do directly in R
[rgraph| 
  require(ggplot2)
  ggplot(diamonds, aes(carat, price)) + geom_point() + theme(aspect.ratio = 1) |]

Assignment also works as usual:


In [ ]:
[rprint| fact <- function(n) if(n == 0) 1 else n * fact(n - 1) 
  fact(10) |]

Call fact again...

In [ ]:
[rprint| fact(10) |]

The embedded R interpreter is stateful, but for global assignment we need to use <<-

In [ ]:
[r| fact <<- function(n) if(n == 0) 1 else n * fact(n - 1) |]

In [ ]:
[rprint| fact(25) |]

## Fine, but my data are in Haskell ... how do I ship them over to R?

The \_hs suffix is used to splice values from Haskell into the R code:

In [ ]:
-- a Haskell list
xs = [1..10] :: [Double]
xs

In [ ]:
[rprint| mean(xs_hs) |]

We can even use functions defined in Haskell in R:

In [10]:
import Data.Int
fact :: Int32 -> Int32
fact 0 = 1
fact n = n * fact (n - 1)

factR x = return $ fact x :: H.R s Int32

In [11]:
n = 6 :: Int32
[rprint| factR_hs(n_hs) |]

[1] 720

## What if I want to use results from R computations in Haskell?

In [ ]:
get_normals :: Double -> R s [Double]
get_normals n = do
   H.dynSEXP <$> [r| rnorm(n_hs) |]
result <- H.runRegion $ get_normals 4

result